## Welcome to a quick demo of GeometricFlux!
Run the cells as you go to view how portions operate. Description boxes are added above each one to help you figure out what is going on!

In [1]:
using GeometricFlux
using Flux
using Flux: onehotbatch, onecold, logitcrossentropy, throttle
using Flux: @epochs
using Flux.Data: DataLoader
using JLD2
using Statistics
using SparseArrays
using LightGraphs.SimpleGraphs
using LightGraphs: adjacency_matrix
using CUDA
using Random
using Plots
using GraphRecipes

We are using a graph dataset held in a JLD2 file. These are stored in a sparsearray datastructure as the number of 0 values is large, and it saves on space as a result.

In [2]:
@load "data/cora_features.jld2" features
@load "data/cora_labels.jld2" labels
@load "data/cora_graph.jld2" g

1-element Vector{Symbol}:
 :g

Define the constants used in the NN. These can be modified depending on the architechture you would like to use

In [4]:
num_features = 1433
hidden = 16
target_catg = 7
epochs = 100

100

Change from sparsearray to a matrix (Adjacency Matrix)

In [3]:
## Preprocessing data
train_X = Matrix{Float32}(features)  # dim: num_features * num_nodes
train_y = Matrix{Float32}(labels)  #dim: target_catg * num_nodes
adj_mat = Matrix{Float32}(adjacency_matrix(g))

2708×2708 Matrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

Quickly visualize the data that we are looking at for demonstration purposes

In [ ]:
graphplot(adj_mat)

In [5]:
train_X

1433×2708 Matrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0

In [6]:
train_y

7×2708 Matrix{Float32}:
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  1.0  1.0  1.0  0.0  0.0
 0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0

Define the model. This is a chain of convolutional layers that I will modify as I follow along the demo. This has the most room for modifications

In [8]:
#### Model
model = Chain(GCNConv(adj_mat, num_features=>hidden, relu),
              Dropout(0.5),
              GCNConv(adj_mat, hidden=>target_catg),
              )


Chain(GCNConv(G(V=2708, E), 1433=>16, relu), Dropout(0.5), GCNConv(G(V=2708, E), 16=>7))

Define a loss function for the model

In [9]:
## Loss
loss(x, y) = logitcrossentropy(model(x), y)
accuracy(x, y) = mean(onecold(softmax(cpu(model(x)))) .== onecold(cpu(y)))

accuracy (generic function with 1 method)

In [10]:
?onecold

search: SkipConnection VariationalEncoder ExponentialBackOff



```
onecold(y[, labels = 1:length(y)])
```

Inverse operations of [`onehot`](@ref).

# Examples

```jldoctest
julia> Flux.onecold([true, false, false], [:a, :b, :c])
:a

julia> Flux.onecold([0.3, 0.2, 0.5], [:a, :b, :c])
:c
```


Train the model, and use a callback throttled to reduce output

In [11]:
## Training
ps = Flux.params(model)
train_data = [(train_X, train_y)]
opt = ADAM(0.01)
evalcb() = @show(accuracy(train_X, train_y))

@epochs epochs Flux.train!(loss, ps, train_data, opt, cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\Mel\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


accuracy(train_X, train_y) = 0.17466765140324964


┌ Info: Epoch 2
└ @ Main C:\Users\Mel\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


accuracy(train_X, train_y) = 0.2189807976366322


┌ Info: Epoch 3
└ @ Main C:\Users\Mel\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


...

accuracy(train_X, train_y) = 0.5081240768094535


┌ Info: Epoch 31
└ @ Main C:\Users\Mel\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


accuracy(train_X, train_y) = 0.5125553914327917
accuracy(train_X, train_y) = 0.516248153618907

┌ Info: Epoch 32
└ @ Main C:\Users\Mel\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


LoadError: InterruptException:

## Lets try a more applicable model on a MOF!
We will use this fun model to take the coordinates of the MOFs (X,Y,Z) and try to figure the charges of each one of the atoms in the system. 

In [ ]:
@load "data/ABAVIJ_clean_forcefield_precursors_feature.jld2" feature
@load "data/ABAVIJ_clean_forcefield_precursors_label.jld2" label
@load "data/ABAVIJ_clean_forcefield_precursors_g.jld2" adj_mat

We currently have vector forms of our data from the conversion beforehand. To work in geometricflux we will use the matrix datatype

In [ ]:
train_X = Matrix{Float32}(feature)  # dim: num_features * num_nodes
train_y = Matrix{Float32}(label)  #dim: target_catg * num_nodes

In [ ]:
train_X

In [ ]:
train_y

Define the parameters of the model, the arguements used to set the laye sizes and epochs

In [ ]:
num_features = 3
hidden = 7
target_catg = 1
epochs = 100

Define the model, we will use GCN conv for simplicities sake

In [ ]:
#### Model
model = Chain(GCNConv(adj_mat, num_features=>hidden, relu),
              Dropout(0.5),
              GCNConv(adj_mat, hidden=>target_catg),
              )


Lets use MSE loss this time, we dont want very large differences

In [ ]:
loss(x, y) = Flux.mse(model(x), y)
accuracy(x, y) = mean(onecold(softmax(cpu(model(x)))) .== onecold(cpu(y)))

Train this just like last time. For smaller numbers, lets take the time to observe the loss overtime instead of the accuracy

In [ ]:
## Training
ps = Flux.params(model)
train_data = [(train_X, train_y)]
opt = ADAM(0.01)
evalcb() = @show(loss(train_X, train_y))

@epochs epochs Flux.train!(loss, ps, train_data, opt, cb=throttle(evalcb, 10))

We can use the model function to see what our output looks like!

In [ ]:
model(train_X)